In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import pickle
import time
from tqdm import tqdm

from gensim.models import Word2Vec
from node2vec import Node2Vec

from sklearn.linear_model import LogisticRegression

from sklearn.metrics.pairwise import cosine_similarity


In [14]:
input_dir = './data/facebook/'
BATCH_COUNT = 8
# node2vec settings
WINDOW_SIZE = 10 # Context size for optimization
NUM_WALKS = 10 # Number of walks per source
WALK_LENGTH = 80 # Length of walk per source
DIMENSIONS = 128 # Embedding dimension
DIRECTED = False # Graph directed/undirected
WORKERS = 6 # Num. parallel workers
ITER = 1 # SGD epochs

In [15]:
def get_edge_embeddings(emb_mappings, edge_list):
    embs = []
    for edge in edge_list:
        node1 = edge[0]
        node2 = edge[1]
        if node1 in emb_mappings:
            emb1 = emb_mappings[node1]
        else:
            continue
        if node2 in emb_mappings:
            emb2 = emb_mappings[node2]
        else:
            continue
        edge_emb = np.multiply(emb1, emb2)
        embs.append(edge_emb)
    embs = np.array(embs)
    return embs


def read_edge_list(file_path):
    with open(file_path) as f:
        return [(line.split(' ')[0],line.split(' ')[1]) for line in f.read().split('\n')[:-1]]
        

def get_combination_model(first, second, verbose=True):
    edge_list_path = input_dir + str(first)+'_'+str(second)+'_edges.txt'
    first_nodes_path = input_dir + str(first) + '_nodes.txt'
    second_nodes_path = input_dir + str(second) + '_nodes.txt'

    first_node2vec = {}
    second_node2vec = {}
    edge_list = []

    with open(edge_list_path) as f:
        edge_list = [line.split(' ') for line in f.read().split('\n')[:-1]]
    edge_list =[(str(min(int(edge[0]), int(edge[1]))), str(max(int(edge[0]), int(edge[1])))) for edge in edge_list]
    for e in edge_list:
        assert int(e[0]) <= int(e[1])

    with open(first_nodes_path) as f:
        for node in f.read().split('\n'):
            if node.strip() != '':
                first_node2vec[node.strip()] = None

    with open(second_nodes_path) as f:
        for node in f.read().split('\n'):
            if node.strip() != '':
                second_node2vec[node.strip()] = None

    all_nodes = list(first_node2vec.keys()) + list(second_node2vec.keys())
    g_train = nx.read_edgelist(edge_list_path)
    start_time = time.time()
    node2vec = Node2Vec(g_train, dimensions=DIMENSIONS, walk_length=WALK_LENGTH, num_walks=NUM_WALKS, workers=WORKERS)
    model = node2vec.fit(window=WINDOW_SIZE, min_count=0, iter=ITER, workers=WORKERS)
    print('node2vec took', time.time() - start_time)
    emb_mappings = model.wv
    #     print('len(emb_mappings)', len(emb_mappings))

    train_edges_false = []
    for e in read_edge_list(input_dir + 'train_edges_false.txt'):
        assert int(e[0]) <= int(e[1])
        if e[0] in emb_mappings and e[1] in emb_mappings:
            train_edges_false.append(e)

    while(len(train_edges_false) < len(edge_list)):
        idx_i = int(all_nodes[np.random.randint(0, len(all_nodes))])
        idx_j = int(all_nodes[np.random.randint(0, len(all_nodes))])

        if idx_i == idx_j:
            continue

        false_edge = (str(min(idx_i, idx_j)), str(max(idx_i, idx_j)))
        idx_i = false_edge[0]
        idx_j = false_edge[1]
        if idx_i not in emb_mappings:
            continue
        if idx_j not in emb_mappings:
            continue
        # Make sure false_edge not an actual edge, and not a repeat
        if false_edge in train_edges_false:
            continue
        if false_edge in edge_list:
            continue

        train_edges_false.append(false_edge)

    for e in train_edges_false:
        assert int(e[0]) <= int(e[1])

    edge_list_set = set(edge_list)
    train_edges_false_set = set(train_edges_false)
    assert len(edge_list_set.intersection(train_edges_false_set)) == 0

    pos_train_edge_embs = get_edge_embeddings(emb_mappings, edge_list)
    neg_train_edge_embs = get_edge_embeddings(emb_mappings, train_edges_false)

    train_edge_embs = np.concatenate([pos_train_edge_embs, neg_train_edge_embs])

    # Create train-set edge labels: 1 = real edge, 0 = false edge
    train_edge_labels = np.concatenate([
        np.ones(pos_train_edge_embs.shape[0]), np.zeros(neg_train_edge_embs.shape[0])
    ])

    assert pos_train_edge_embs.shape[0] == neg_train_edge_embs.shape[0]

    # if verbose:
    #     print(pos_train_edge_embs.shape, neg_train_edge_embs.shape, train_edge_labels.shape)

    edge_classifier = LogisticRegression(random_state=0, solver='lbfgs', max_iter=250)
    edge_classifier.fit(train_edge_embs, train_edge_labels)

    for key in first_node2vec.keys():
        if key in emb_mappings:
            first_node2vec[key] = emb_mappings[key]
        else:
            first_node2vec[key] = np.zeros((128,))

    for key in second_node2vec.keys():
        if key in emb_mappings:
            second_node2vec[key] = emb_mappings[key]
        else:
            second_node2vec[key] = np.zeros((128,))
    
    emb_mappings = {}
    for key in first_node2vec.keys():
        emb_mappings[key] = first_node2vec[key]
    for key in second_node2vec.keys():
        emb_mappings[key] = second_node2vec[key]
    return {
        'model': edge_classifier,
        'first_node2vec': first_node2vec,
        'second_node2vec': second_node2vec,
        'emb_mappings': emb_mappings
    }

In [16]:
combinations = {}

In [17]:
combinations[(0,1)] = get_combination_model(0,1)

Generating walks (CPU: 4): 100%|██████████| 2/2 [00:14<00:00,  7.35s/it]


node2vec took 30.559221982955933


In [18]:
combinations[(1,2)] = get_combination_model(1,2)

Generating walks (CPU: 4): 100%|██████████| 2/2 [00:14<00:00,  7.26s/it]


node2vec took 27.757361888885498


In [19]:
combinations[(2,3)] = get_combination_model(2,3)

Generating walks (CPU: 4): 100%|██████████| 2/2 [00:14<00:00,  7.29s/it]


node2vec took 26.147933959960938


In [ ]:
batch_nodes = []
for batch_idx in range(BATCH_COUNT):
    batch_nodes.append([])
    
for key, value in combinations.items():
    batch_nodes[key[0]] = list(value['first_node2vec'].keys())
    batch_nodes[key[1]] = list(value['second_node2vec'].keys())

In [ ]:
parts_graph = nx.Graph()
parts_graph.add_edge(0,1)
parts_graph.add_edge(1,2)
parts_graph.add_edge(2,3)

In [ ]:
test_edge_true = read_edge_list(input_dir + 'test_edges_true.txt')
test_edge_false= read_edge_list(input_dir + 'test_edges_false.txt')
test_edges = []
for e in test_edge_true:
    test_edges.append((e[0],e[1],1))
for e in test_edge_false:
    test_edges.append((e[0],e[1],0))

In [ ]:
def get_batch_idx(node):
    for batch_idx in range(BATCH_COUNT):
        if node in batch_nodes[batch_idx]:
            return batch_idx
    assert False
    
def get_combination(b1,b2=None):
    if b2 == None:
        for comb in combinations.keys():
            if b1 in comb:
                return comb
    else:
        for comb in combinations.keys():
            if (b1,b2) == comb:
                return comb
    
    
distances0 = 0
distances1 = 0
distances2 = 0
for e in test_edges:
    batch_x = get_batch_idx(e[0])
    batch_y = get_batch_idx(e[1])
    distance = len(nx.shortest_path(parts_graph, batch_x, batch_y))
    if distance == 2 or distance == 1:
        distances0 += 1
    elif distance == 3:
        distances1 += 1
    elif distance == 4:
        distances2 += 1
    else:
        print(distance, nx.shortest_path(parts_graph, batch_x, batch_y))
print(distances0 + distances1 + distances2, len(test_edges))

In [ ]:
test_edge_embs = []
test_edge_lbls = []
test_edge_pred = []

for e in tqdm(test_edges):
    batch_x = min(get_batch_idx(e[0]),get_batch_idx(e[1]))
    batch_y = max(get_batch_idx(e[0]),get_batch_idx(e[1]))
    
    path = nx.shortest_path(parts_graph, batch_x, batch_y)
    
    if len(path) == 1:
        comb_name = get_combination(path[0])
        values = combinations[comb_name]
        edge_emb = np.multiply(
            values['emb_mappings'][e[0]], 
            values['emb_mappings'][e[1]]
        )
        pred = values['model'].predict_proba([edge_emb])[:, 1]
    elif len(path) == 2:
        comb_name = get_combination(path[0], path[1])
        values = combinations[comb_name]
        edge_emb = np.multiply(
            values['emb_mappings'][e[0]], 
            values['emb_mappings'][e[1]]
        )
        pred = values['model'].predict_proba([edge_emb])[:, 1]
    if len(path) <= 2:    
        test_edge_embs.append(edge_emb)
        test_edge_lbls.append(e[2])
        test_edge_pred.append(pred)
    
test_roc = roc_auc_score(test_edge_lbls, test_edge_pred)
test_ap = average_precision_score(test_edge_lbls, test_edge_pred)
print(len(test_edge_lbls),'out of', len(test_edges))
print ('node2vec Test ROC score: ', str(test_roc))
print ('node2vec Test AP score: ', str(test_ap))    

In [ ]:
def get_similar_in_other_half(values, node):
    lookforin = None
    node_vector = values['emb_mappings'][node]
    
    if node in values['first_node2vec']:
        lookforin = values['second_node2vec']
    elif node in values['second_node2vec']:
        lookforin = values['first_node2vec']
    
    node_name = None
    distance = 10000
    for other_node, other_vector in lookforin.items():
        dis =  np.linalg.norm(other_vector - node_vector)
        if dis < distance:
            distance = dis
            node_name = other_node
    return node_name, values['emb_mappings'][node_name]
    
test_edge_embs = []
test_edge_lbls = []
test_edge_pred = []

for e in tqdm(test_edges):
    batch_x = min(get_batch_idx(e[0]),get_batch_idx(e[1]))
    batch_y = max(get_batch_idx(e[0]),get_batch_idx(e[1]))
    
    path = nx.shortest_path(parts_graph, batch_x, batch_y)
    
    if len(path) != 3:
        continue
    if e[0] in batch_nodes[path[0]]:
        alpha = e[0]
        theta = e[1]
        alpha_embeding = combinations[(path[0],path[1])]['emb_mappings'][alpha]
        theta_embeding = combinations[(path[1],path[2])]['emb_mappings'][theta]
    else:
        alpha = e[1]
        theta = e[0]
        alpha_embeding = combinations[(path[1],path[2])]['emb_mappings'][alpha]
        theta_embeding = combinations[(path[0],path[1])]['emb_mappings'][theta]
        
    
#     alpha_prim = the most similar one to alpha in path[1] & (path[0],path[1])
    alpha_prime, alpha_prime_vec = get_similar_in_other_half(combinations[(path[0],path[1])], alpha)
    alpha_prime_in_B_vec = combinations[(path[1],path[2])]['emb_mappings'][alpha_prime]
    
#     theta_prim = the most similar one to theta in path[1] & (path[1],path[2])
    theta_prim, theta_prim_vec = get_similar_in_other_half(combinations[(path[1],path[2])], theta)
    
    
    
#     there would be 2 predictions: 
#         one using (path[0],path[1]) , (alpha, theta_prime)
#         one using (path[1],path[2]) , (theta, alpha_prime)
#     the average of these 2 is my opinion
    
#     if len(path) == 3:
    comb_name = (path[1],path[2])
    values = combinations[comb_name]
    edge_emb = np.multiply(
        alpha_prime_in_B_vec, 
        theta_embeding
    )
    pred = values['model'].predict_proba([edge_emb])[:, 1]

    test_edge_embs.append(edge_emb)
    test_edge_lbls.append(e[2])
    test_edge_pred.append(pred)

test_roc = roc_auc_score(test_edge_lbls, test_edge_pred)
test_ap = average_precision_score(test_edge_lbls, test_edge_pred)

print(len(test_edge_lbls),'out of', len(test_edges))
print ('node2vec Test ROC score: ', str(test_roc))
print ('node2vec Test AP score: ', str(test_ap))

In [ ]:
def get_similar_in_other_half(values, node):
    lookforin = None
    node_vector = values['emb_mappings'][node]
    
    if node in values['first_node2vec']:
        lookforin = values['second_node2vec']
    elif node in values['second_node2vec']:
        lookforin = values['first_node2vec']
    
    node_name = None
    distance = 10000
    for other_node, other_vector in lookforin.items():
        dis =  np.linalg.norm(other_vector - node_vector)
        if dis < distance:
            distance = dis
            node_name = other_node
    return node_name, values['emb_mappings'][node_name]
    
test_edge_embs = []
test_edge_lbls = []
test_edge_pred = []

for e in tqdm(test_edges):
    batch_x = min(get_batch_idx(e[0]),get_batch_idx(e[1]))
    batch_y = max(get_batch_idx(e[0]),get_batch_idx(e[1]))
    
    path = nx.shortest_path(parts_graph, batch_x, batch_y)
    
    if len(path) != 4:
        continue
    if e[0] in batch_nodes[path[0]]:
        alpha = e[0]
        theta = e[1]
        alpha_embeding = combinations[(path[0],path[1])]['emb_mappings'][alpha]
        theta_embeding = combinations[(path[2],path[3])]['emb_mappings'][theta]
    else:
        alpha = e[1]
        theta = e[0]
        alpha_embeding = combinations[(path[2],path[3])]['emb_mappings'][alpha]
        theta_embeding = combinations[(path[0],path[1])]['emb_mappings'][theta]
    
#     alpha_prim = the most similar one to alpha in path[1] & (path[0],path[1])
    alpha_prime, alpha_prime_vec = get_similar_in_other_half(combinations[(path[0],path[1])], alpha)
    alpha_prime_in_12_vec = combinations[(path[1],path[2])]['emb_mappings'][alpha_prime]
    
#     theta_prim = the most similar one to theta in path[2] & (path[2],path[3])
    theta_prim, theta_prim_vec = get_similar_in_other_half(combinations[(path[2],path[3])], theta)
    theta_prime_in_12_vec = combinations[(path[1],path[2])]['emb_mappings'][theta_prim]
    
    comb_name = (path[1],path[2])
    values = combinations[comb_name]
    edge_emb = np.multiply(
        alpha_prime_in_12_vec, 
        theta_prime_in_12_vec
    )
    pred = values['model'].predict_proba([edge_emb])[:, 1]

    test_edge_embs.append(edge_emb)
    test_edge_lbls.append(e[2])
    test_edge_pred.append(pred)

test_roc = roc_auc_score(test_edge_lbls, test_edge_pred)
test_ap = average_precision_score(test_edge_lbls, test_edge_pred)

print(len(test_edge_lbls),'out of', len(test_edges))
print ('node2vec Test ROC score: ', str(test_roc))
print ('node2vec Test AP score: ', str(test_ap))

In [ ]:
0.9751715475554089 * (13276/52940) + 0.9924898041533882 * (33033/52940) + 0.9704944447856797 * (6631/52940)

In [ ]:
33033 + 13276 + 6631